In [1]:
import numpy as np
import numpy.random as rgt
from scipy.stats import norm, t
import matplotlib.pyplot as plt
import time
import pandas as pd

from conquer.linear_model import low_dim
rgt.seed(1)

The reference machine used for the simulations below is a Mac Pro with a 3.2 GHz 16-Core Intel Xeon processor and 96 GB of RAM. Each simulation exploits only one processor, without any parallelism.

In [2]:
n, p = 8000, 400
itcp, beta = 4, 1*np.ones(p)*(2*rgt.binomial(1, 1/2, p) - 1)
tau, t_df = 0.75, 2
runtime = 0

M = 100
itcp_se, coef_se = np.empty(M), np.empty(M)
for m in range(M):
    X = rgt.normal(0, 1.5, size=(n,p))
    Y = itcp + X.dot(beta) + rgt.standard_t(t_df, n) - t.ppf(tau, t_df)

    tic = time.time()
    model = low_dim(X, Y).fit(tau=tau)
    runtime += time.time() - tic

    itcp_se[m] = (model['beta'][0] - itcp)**2
    coef_se[m] = np.sum((model['beta'][1:] - beta)**2)

In [3]:
out = {'MSE (itcp)': np.mean(itcp_se), 
       'std (itcp)': np.std(itcp_se), 
       'MSE (coef)': np.mean(coef_se),
       'std (coef)': np.std(coef_se),
       'Runtime': runtime/M}
out = pd.DataFrame(out, index=['conquer'])
out

,MSE (itcp),std (itcp),MSE (coef),std (coef),Runtime
conquer,0.002074,0.001918,0.07636,0.006175,0.048587


### Construction of confidence intervals

In [4]:
n, p = 500, 20
mask = 2*rgt.binomial(1, 1/2, p) - 1
itcp, beta = 4, 1*np.ones(p)*mask
tau, t_df = 0.75, 2

M = 500
ci_cover = np.zeros([4, p])
ci_width = np.empty([M, 4, p])
for m in range(M):
    X = rgt.normal(0, 1.5, size=(n,p))
    Y = itcp + X.dot(beta) + rgt.standard_t(t_df, n) - t.ppf(tau, t_df)

    sqr = low_dim(X, Y)    
    model1 = sqr.norm_ci(tau)
    model2 = sqr.mb_ci(tau)
    
    ci_cover[0,:] += (beta >= model1['normal_ci'][1:,0])*(beta<= model1['normal_ci'][1:,1])
    ci_cover[1,:] += (beta >= model2['percentile_ci'][1:,0])*(beta<= model2['percentile_ci'][1:,1])
    ci_cover[2,:] += (beta >= model2['pivotal_ci'][1:,0])*(beta<= model2['pivotal_ci'][1:,1])
    ci_cover[3,:] += (beta >= model2['normal_ci'][1:,0])*(beta<= model2['normal_ci'][1:,1])
    
    ci_width[m,0,:] = model1['normal_ci'][1:,1] - model1['normal_ci'][1:,0]
    ci_width[m,1,:] = model2['percentile_ci'][1:,1] - model2['percentile_ci'][1:,0]
    ci_width[m,2,:] = model2['pivotal_ci'][1:,1] - model2['pivotal_ci'][1:,0]
    ci_width[m,3,:] = model2['normal_ci'][1:,1] - model2['normal_ci'][1:,0]

In [5]:
cover = pd.DataFrame(ci_cover/M, index=["Normal", "MB-Percentile", "MB-Pivotal", "MB-Normal"])
cover.columns = pd.Index(np.linspace(1,p,p), dtype=int)
cover

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Normal,0.956,0.970,0.976,0.976,0.972,0.980,0.972,0.954,0.954,0.976,0.960,0.960,0.978,0.956,0.972,0.964,0.962,0.938,0.956,0.976
MB-Percentile,0.970,0.962,0.972,0.974,0.976,0.972,0.976,0.944,0.960,0.976,0.970,0.958,0.960,0.940,0.962,0.968,0.962,0.948,0.954,0.976
MB-Pivotal,0.932,0.928,0.954,0.952,0.928,0.962,0.948,0.916,0.920,0.936,0.918,0.912,0.936,0.898,0.946,0.936,0.930,0.888,0.934,0.948
MB-Normal,0.946,0.966,0.976,0.976,0.964,0.978,0.980,0.944,0.960,0.972,0.958,0.956,0.964,0.952,0.966,0.966,0.952,0.932,0.958,0.970


In [6]:
width = pd.DataFrame(np.mean(ci_width, axis=0), index=["Normal", "MB-Percentile", "MB-Pivotal", "MB-Normal"])
width.columns = cover.columns
width

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Normal,0.258618,0.255194,0.258960,0.257864,0.256178,0.254467,0.262488,0.255332,0.257358,0.253531,0.253908,0.256757,0.254476,0.255028,0.257954,0.254567,0.256084,0.257422,0.254076,0.254236
MB-Percentile,0.225053,0.223851,0.224437,0.226008,0.222294,0.223036,0.225199,0.224579,0.224054,0.222668,0.224079,0.223610,0.221909,0.223040,0.222418,0.222918,0.222833,0.223637,0.222723,0.221489
MB-Pivotal,0.225053,0.223851,0.224437,0.226008,0.222294,0.223036,0.225199,0.224579,0.224054,0.222668,0.224079,0.223610,0.221909,0.223040,0.222418,0.222918,0.222833,0.223637,0.222723,0.221489
MB-Normal,0.228839,0.227647,0.228186,0.229298,0.226129,0.225956,0.229295,0.227726,0.228473,0.226464,0.227731,0.227835,0.225931,0.227478,0.226373,0.227070,0.226769,0.228031,0.226377,0.225124


# Heteroscedastic model

Let $z=(z_1, \ldots, z_p)^T \sim N(0, \Sigma)$ with $\Sigma = (0.5^{|j-k|})_{1\leq j, k \leq p}$ and $z_0 \sim {\rm Unif}(0,2)$ be independent. Generate independent data vectors $\{(y_i , x_i) \}_{i=1}^n$ from the model 
$$
    y_i =  \varepsilon_i x_{i1}  +  x_{i2} + \cdots + x_{ip}   \quad {\rm with } \ \  x_i = (x_{i1}, \ldots, x_{ip})^T \sim (z_0, z_2, \ldots, z_p)^T,
$$
where $\varepsilon_i$'s are iid $N(0,1)$ variables that are independent of $x_i$'s.

Consider two quantile levels: $\tau=0.5$ and $\tau=0.8$. Note that the effect of $x_{i1}$ is only present for $\tau=0.8$.

In [7]:
def cov_generate(std, corr=0.5):
    p = len(std)
    R = np.zeros(shape=[p,p])
    for j in range(p-1):
        R[j, j+1:] = np.array(range(1, len(R[j,j+1:])+1))
    R += R.T
    return np.outer(std, std) * (corr*np.ones(shape=[p,p]))** R
        
n = 2000
p = 10
mu, Sig = np.zeros(p), cov_generate(np.ones(p), 0.5)
beta = np.ones(p)
beta[0] = 0

### Case 1: $\tau=0.5$.
The conditional median of $y_i$ given $x_i$ is $Q_{0.5}(y_i | x_i) =  x_{i2} + \cdots + x_{ip}$.

In [8]:
tau = 0.5
M = 200
ci_cover = np.zeros([4, p])
ci_width = np.empty([M, 4, p])
for m in range(M):
    X = rgt.multivariate_normal(mean=mu, cov=Sig, size=n)
    X[:,0] = rgt.uniform(0, 2, size=n)
    Y = X.dot(beta) +  X[:,0]*rgt.normal(0,1,size=n)

    sqr = low_dim(X, Y, intercept=False)    
    model1 = sqr.norm_ci(tau)
    model2 = sqr.mb_ci(tau)
    
    ci_cover[0,:] += (beta >= model1['normal_ci'][:,0])*(beta<= model1['normal_ci'][:,1])
    ci_cover[1,:] += (beta >= model2['percentile_ci'][:,0])*(beta<= model2['percentile_ci'][:,1])
    ci_cover[2,:] += (beta >= model2['pivotal_ci'][:,0])*(beta<= model2['pivotal_ci'][:,1])
    ci_cover[3,:] += (beta >= model2['normal_ci'][:,0])*(beta<= model2['normal_ci'][:,1])
    
    ci_width[m,0,:] = model1['normal_ci'][:,1] - model1['normal_ci'][:,0]
    ci_width[m,1,:] = model2['percentile_ci'][:,1] - model2['percentile_ci'][:,0]
    ci_width[m,2,:] = model2['pivotal_ci'][:,1] - model2['pivotal_ci'][:,0]
    ci_width[m,3,:] = model2['normal_ci'][:,1] - model2['normal_ci'][:,0]

In [9]:
cover = pd.DataFrame(ci_cover/M, index=["Normal", "MB-Percentile", "MB-Pivotal", "MB-Normal"])
cover.columns = pd.Index(np.linspace(1,p,p), dtype=int)
cover

,1,2,3,4,5,6,7,8,9,10
Normal,0.94,0.960,0.975,0.970,0.960,0.960,0.97,0.940,0.98,0.955
MB-Percentile,0.94,0.955,0.980,0.945,0.945,0.940,0.97,0.945,0.97,0.910
MB-Pivotal,0.92,0.985,0.995,0.975,0.960,0.955,0.98,0.965,0.99,0.980
MB-Normal,0.94,0.970,0.995,0.970,0.970,0.965,0.98,0.950,0.99,0.960


In [10]:
width = pd.DataFrame(np.mean(ci_width, axis=0), index=["Normal", "MB-Percentile", "MB-Pivotal", "MB-Normal"])
width.columns = cover.columns
width

,1,2,3,4,5,6,7,8,9,10
Normal,0.124566,0.062799,0.070737,0.070428,0.070386,0.070730,0.070815,0.070838,0.069831,0.062915
MB-Percentile,0.120680,0.064697,0.073046,0.072585,0.073018,0.072725,0.072837,0.072483,0.072315,0.065179
MB-Pivotal,0.120680,0.064697,0.073046,0.072585,0.073018,0.072725,0.072837,0.072483,0.072315,0.065179
MB-Normal,0.123163,0.065584,0.074020,0.074022,0.074079,0.073809,0.073901,0.073775,0.073308,0.066165


### Case 2: $\tau=0.8$. 
In this case, the conditional $0.8$-quantile of $y_i$ given $x_i$ is $Q_{0.8}(y_i | x_i) =   \Phi^{-1}(0.8) x_{i1} + x_{i2} + \cdots + x_{ip}$.

In [11]:
tau = 0.8
true_beta = np.copy(beta)
true_beta[0] = norm.ppf(tau)

M = 200
ci_cover = np.zeros([4, p])
ci_width = np.empty([M, 4, p])
for m in range(M):
    X = rgt.multivariate_normal(mean=mu, cov=Sig, size=n)
    X[:,0] = rgt.uniform(0, 2, size=n)
    Y = X.dot(beta) + X[:,0]*rgt.normal(0,1,size=n)

    sqr = low_dim(X, Y, intercept=False)    
    model1 = sqr.norm_ci(tau)
    model2 = sqr.mb_ci(tau)
    
    ci_cover[0,:] += (true_beta>=model1['normal_ci'][:,0])*(true_beta<= model1['normal_ci'][:,1])
    ci_cover[1,:] += (true_beta>=model2['percentile_ci'][:,0])*(true_beta<= model2['percentile_ci'][:,1])
    ci_cover[2,:] += (true_beta>=model2['pivotal_ci'][:,0])*(true_beta<= model2['pivotal_ci'][:,1])
    ci_cover[3,:] += (true_beta>=model2['normal_ci'][:,0])*(true_beta<= model2['normal_ci'][:,1])
    
    ci_width[m,0,:] = model1['normal_ci'][:,1] - model1['normal_ci'][:,0]
    ci_width[m,1,:] = model2['percentile_ci'][:,1] - model2['percentile_ci'][:,0]
    ci_width[m,2,:] = model2['pivotal_ci'][:,1] - model2['pivotal_ci'][:,0]
    ci_width[m,3,:] = model2['normal_ci'][:,1] - model2['normal_ci'][:,0]
        
cover = pd.DataFrame(ci_cover/M, index=["Normal", "MB-Percentile", "MB-Pivotal", "MB-Normal"])
cover.columns = pd.Index(np.linspace(1,p,p), dtype=int)

width = pd.DataFrame(np.mean(ci_width, axis=0), index=["Normal", "MB-Percentile", "MB-Pivotal", "MB-Normal"])
width.columns = cover.columns

In [12]:
cover

,1,2,3,4,5,6,7,8,9,10
Normal,0.950,0.935,0.970,0.970,0.935,0.96,0.965,0.950,0.955,0.945
MB-Percentile,0.955,0.935,0.965,0.965,0.935,0.95,0.965,0.945,0.950,0.930
MB-Pivotal,0.935,0.945,0.975,0.975,0.950,0.96,0.980,0.960,0.970,0.955
MB-Normal,0.955,0.950,0.985,0.975,0.950,0.97,0.985,0.955,0.960,0.955


In [13]:
width

,1,2,3,4,5,6,7,8,9,10
Normal,0.142014,0.065565,0.072919,0.072517,0.073149,0.072836,0.073099,0.072850,0.073250,0.064128
MB-Percentile,0.138092,0.067997,0.075980,0.075077,0.074973,0.076113,0.076349,0.075781,0.075694,0.066715
MB-Pivotal,0.138092,0.067997,0.075980,0.075077,0.074973,0.076113,0.076349,0.075781,0.075694,0.066715
MB-Normal,0.140658,0.069106,0.077041,0.076165,0.076402,0.077219,0.077378,0.076721,0.077102,0.067611
